In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tabulate import tabulate
import plotly.express as px

### Variables & Parameters

In [2]:
from ortools.linear_solver import pywraplp

# Create the solver with the GLOP solver
solver = pywraplp.Solver.CreateSolver('GLOP')

In [3]:
# Single value parameters
pmax = 500  #m3/s
qmax = 30  #m3/s
vmax = 500  #hm3
v0 = 100  #hm3
C = 0.0864  #m3/s to hm3/day
N = 10  #number of time steps
infinity = solver.infinity()  # simplify writing

# Vector parameters
eff = [1.00, 0.05, 0.1, 0.15, 0.20, 0.25, 0.80, 2.00, 2.00, 1.00]  # created, not from height
a = np.array([1, 2, 4, 8, 16, 16, 8, 4, 2, 1]) * 10  # scaled arbitrarily to create interesting model

# External variables
p = {}  # p[t] is the flow through the turbines at time t
q = {}  # q[t] is the flow dumped in the lake at time t

# Implicit variables
v = {}  # v[t] is the volume in reservoir at time t, hm3
prod = {}  # prod[t] is the energy production, kWh | prod[t] = p[t]*eff[t]

In [4]:
# Create variables, N+1 to accommodate for v[t+1] term
for t in range(N + 1):
    p[t] = solver.NumVar(0, infinity, 'p[%i]' % t)
    q[t] = solver.NumVar(0, infinity, 'q[%i]' % t)
    v[t] = solver.NumVar(0, infinity, 'v[%i]' % t)
    prod[t] = solver.NumVar(0, infinity, 'prod[%i]' % t)

print('Number of variables =', solver.NumVariables())

Number of variables = 44


In [5]:
for t in range(N):
    # initial conditions
    solver.Add(v[0] == v0)  # initial volume

    # constraints that contain [t-1] terms
    if t > 0:
        solver.Add(v[t] == v[t - 1] + (C * (a[t - 1] - p[t - 1] - q[t - 1])))  # volume balance
        solver.Add(p[t - 1] - p[t] <= 0.5 * pmax)
        solver.Add(p[t] - p[t - 1] <= 0.5 * pmax)

    # for v[10] term, final volume after last time step
    solver.Add(v[N] == v[N - 1] + (C * (a[N - 1] - p[N - 1] - q[N - 1])))  # volume balance

    # production from efficiency and flow in turbine
    solver.Add(prod[t] == eff[t] * p[t])

    # ceiling constraints
    solver.Add(v[t] <= vmax)
    solver.Add(p[t] <= pmax)
    solver.Add(q[t] <= qmax)

    # non-negativity constraints
    solver.Add(v[t] >= 0)
    solver.Add(p[t] >= 0)
    solver.Add(q[t] >= 0)
    solver.Add(prod[t] >= 0)

In [6]:
# Objective function
objective = solver.Objective()
# equilvalent to sum(prod[t] for t in range(N)) or prod[0]+prod[1]+...+prod[N-1]
for t in range(N):
    objective.SetCoefficient(prod[t], 1)
objective.SetMaximization()
#for t in range(N):
#    obj_expr = [prod[t]]
#    solver.Maximize(solver.Sum(obj_expr))

In [7]:
# Solve
status = solver.Solve()

In [8]:
# Extract solution
p_sol = []
q_sol = []
v_sol = []
prod_sol = []

# If optimality is reached, print the solution
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for t in range(N + 1):
        print(p[t].name(), ' = ', p[t].solution_value())
        print(q[t].name(), ' = ', q[t].solution_value())
        print(v[t].name(), ' = ', v[t].solution_value())
        p_sol.append(p[t].solution_value())
        q_sol.append(q[t].solution_value())
        v_sol.append(v[t].solution_value())
        prod_sol.append(prod[t].solution_value())
        print()

Objective value = 2727.4074074074074
p[0]  =  250.00000000000003
q[0]  =  0.0
v[0]  =  100.0

p[1]  =  0.0
q[1]  =  0.0
v[1]  =  79.264

p[2]  =  0.0
q[2]  =  0.0
v[2]  =  80.99199999999999

p[3]  =  0.0
q[3]  =  0.0
v[3]  =  84.44800000000001

p[4]  =  0.0
q[4]  =  0.0
v[4]  =  91.36000000000001

p[5]  =  0.0
q[5]  =  0.0
v[5]  =  105.18400000000001

p[6]  =  250.00000000000003
q[6]  =  0.0
v[6]  =  119.008

p[7]  =  500.0
q[7]  =  0.0
v[7]  =  104.32000000000001

p[8]  =  500.0
q[8]  =  0.0
v[8]  =  64.57600000000002

p[9]  =  277.4074074074075
q[9]  =  0.0
v[9]  =  23.10400000000001

p[10]  =  0.0
q[10]  =  0.0
v[10]  =  0.0



In [16]:
df = pd.DataFrame(data=[p_sol, q_sol, a, v_sol, prod_sol, eff]).T
df.columns = ['p_t', 'q_t', 'a_t', 'v_t', 'prod_t', 'eff']
df = df.round(2)
df.style.format(precision=2)
print(tabulate(df, headers=df.columns))
# print all column in plotly line graph
fig = px.line(df, x=df.index, y=df.columns)
fig.show()

       p_t    q_t    a_t     v_t    prod_t     eff
--  ------  -----  -----  ------  --------  ------
 0  250         0     10  100       250       1
 1    0         0     20   79.26      0       0.05
 2    0         0     40   80.99      0       0.1
 3    0         0     80   84.45      0       0.15
 4    0         0    160   91.36      0       0.2
 5    0         0    160  105.18      0       0.25
 6  250         0     80  119.01    200       0.8
 7  500         0     40  104.32   1000       2
 8  500         0     20   64.58   1000       2
 9  277.41      0     10   23.1     277.41    1
10    0         0    nan    0         0     nan


In [11]:
# Simple mathplotlib implementation if no libraries
# plt.plot(p_sol)
# plt.plot(q_sol)
# plt.plot(a)
# plt.plot(v_sol)
# plt.plot(prod_sol)
# plt.plot(np.ones(10) * vmax)
# plt.legend(['p', 'q', 'a', 'v', 'prod', 'vmax'])
# plt.show()